**Load libraries**

In [41]:
import pandas as pd
import seaborn as sns
import numpy as np

**load and read the dataset**

In [42]:
train=pd.read_csv("/kaggle/input/credit-scores/train.csv")
test=pd.read_csv("/kaggle/input/credit-scores/test.csv")
# sample_sub=pd.read_csv("dataset/sample_submission.csv")

In [43]:
train.shape

(25000, 5480)

In [44]:
test.shape

(5000, 5479)

## Preprocessing

**Check for null values**


In [45]:
train.isnull().sum()

id              0
var_1        2124
var_2        2124
var_3        2124
var_4        2124
            ...  
var_5475    23974
var_5476    24211
var_5477    24993
var_5478    22430
target          0
Length: 5480, dtype: int64

In [46]:
train.dropna(axis="columns",thresh=22000,inplace=True)

In [47]:
train.isnull().sum()

id             0
var_1       2124
var_2       2124
var_3       2124
var_4       2124
            ... 
var_5454      23
var_5456    1750
var_5460     208
var_5466    2398
target         0
Length: 2511, dtype: int64

In [48]:
y=train['target']

In [49]:
x=train.drop(["target","id"],axis=1)

In [50]:
x_test=test[x.columns]

In [51]:
x_test.shape

(5000, 2509)

In [52]:
x.shape

(25000, 2509)

**Solve the problem of Naan value**

In [53]:
x.replace([np.inf, -np.inf], np.nan, inplace=True)
x.fillna(x.median(), inplace=True)

In [54]:
x_test.replace([np.inf, -np.inf], np.nan, inplace=True)
x_test.fillna(x_test.median(), inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [55]:
# x_test=test[x.columns]

In [56]:
x.shape

(25000, 2509)

In [57]:
x_test.shape

(5000, 2509)

In [58]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x)
x= scaler.transform(x)

In [59]:
from sklearn.feature_selection import VarianceThreshold
# Assuming X is your feature matrix
selector = VarianceThreshold()
x= selector.fit_transform(x)
x_test=selector.transform(x_test)

/opt/conda/lib/python3.7/site-packages/sklearn/base.py:444: UserWarning: X has feature names, but VarianceThreshold was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [60]:
x.shape

(25000, 706)

In [61]:
x_test.shape

(5000, 706)

**Solve the problem of imbalanced data**

In [62]:
# from collections import Counter

In [63]:
# from imblearn.under_sampling import NearMiss 
# undersampling=NearMiss(version=3,n_neighbors_ver3=3)
# #transform the dataset
# x,y=undersampling.fit_resample(x,y)
# #summarize the new class distribution
# c=Counter(y)
# print(c)

In [64]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight = 'balanced',classes = np.unique(y),y=y)
class_weights = {i : x for i,x in enumerate(class_weights)}

**Modeling**

In [65]:
from sklearn.ensemble import AdaBoostClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, accuracy_score, make_scorer

In [66]:
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve,auc
from sklearn.model_selection import StratifiedKFold
model=AdaBoostClassifier()
kfold=StratifiedKFold(n_splits=10,random_state=1,shuffle=True)
cv_results = cross_val_score(model, x,y, cv=kfold,scoring="accuracy")
model.fit(x,y)

AdaBoostClassifier()

In [67]:
y_pred =model.predict(x_test)
print(metrics.roc_auc_score(y[:5000], y_pred))
print(model.score(x,y))

0.5000904479230895
0.85536


**For Submittion**

In [68]:
probability=model.predict_proba(x_test)[:,1]


In [69]:
probability.shape

(5000,)

In [70]:
submit_sample = pd.DataFrame({
    # Create a column named "id_" that contains a list of IDs in the format "id_0", "id_1", "id_2", etc.
    "id": ["id_" + str(i) for i in range(1,len(y_pred)+1)],
    # Create a column named "predicted" that contains the values of the "pred" array
    "predicted": y_pred,
    # Create a column named "predicted_probability" that contains the values of the "pred_proba" array
    "predicted_probability": probability
})

In [71]:
submit_sample.to_csv("submission.csv",index=False)